In [1]:
from torchvision.datasets import CelebA
import os
import torch.nn as nn
import glob
import os
import torch
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
import torch
import math
# main libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.nn import functional as F
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset
import math
import torchvision.transforms.functional as TF
import cv2
from keras.utils import image_dataset_from_directory
from torchvision import datasets, transforms
from torch.utils.data import random_split
from torch.nn.parameter import Parameter

from torch.nn import init
from torchvision.utils import make_grid
from collections import OrderedDict

import torch
from torch.utils.data import Dataset

from skimage import transform
import skimage.io as io
import numpy as np
from torchvision.utils import save_image

In [2]:
import torch
from torch import nn
from torchvision.models import vgg16
from pathlib import Path
from torch.autograd import Variable

# VGG

In [3]:
class VGG(nn.Module):
    '''
    VGG model
    '''

    def __init__(self, features):
        super(VGG, self).__init__()
        self.features = features

        # placeholder for the gradients
        self.gradients = None
        self.activation = None

    # hook for the gradients of the activations
    def activations_hook(self, grad):
        self.gradients = grad

    def forward(self, x, target_layer=11):
        result = []
        for i in range(len(nn.ModuleList(self.features))):
            x = self.features[i](x)
            if i == target_layer:
                self.activation = x
                h = x.register_hook(self.activations_hook)
            if i == 2 or i == 5 or i == 8 or i == 11 or i == 14 or i == 17 or i == 20 or i == 23 or i == 26 or i == 29 or i == 32 or i == 35 or i == 38:
                result.append(x)

        return result

    def get_activations_gradient(self):
        return self.gradients

    def get_activations(self, x):
        return self.activation

# VGG16

In [3]:
features = list(vgg16('vgg16-397923af.pth').features)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\khali/.cache\torch\hub\checkpoints\vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "B:\Anaconda_install_path\envs\deeplearning\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\khali\AppData\Local\Temp/ipykernel_12144/909665244.py", line 1, in <module>
    features = list(vgg16('vgg16-397923af.pth').features)
  File "B:\Anaconda_install_path\envs\deeplearning\lib\site-packages\torchvision\models\vgg.py", line 162, in vgg16
    return _vgg('vgg16', 'D', False, pretrained, progress, **kwargs)
  File "B:\Anaconda_install_path\envs\deeplearning\lib\site-packages\torchvision\models\vgg.py", line 99, in _vgg
    state_dict = load_state_dict_from_url(model_urls[arch],
  File "B:\Anaconda_install_path\envs\deeplearning\lib\site-packages\torch\hub.py", line 586, in load_state_dict_from_url
    download_url_to_file(url, cached_file, hash_prefix, progress=progress)
  File "B:\Anaconda_install_path\envs\deeplearning\lib\site-packages\torch\hu

TypeError: object of type 'NoneType' has no len()

In [ ]:
X.shape

In [ ]:
features[1](X).shape

In [ ]:
class Vgg16(torch.nn.Module):
    def __init__(self, pretrain):
        super(Vgg16, self).__init__()
        features = list(vgg16('vgg16-397923af.pth').features)

        if not pretrain:
            for ind, f in enumerate(features):
                # nn.init.xavier_normal_(f)
                if type(f) is torch.nn.modules.conv.Conv2d:
                    torch.nn.init.xavier_uniform(f.weight)
                    print("Initialized", ind, f)
                else:
                    print("Bypassed", ind, f)
            # print("Pre-trained Network loaded")
        self.features = nn.ModuleList(features).eval()
        self.output = []

    def forward(self, x):
        output = []
        for i in range(31):
            x = self.features[i](x)
            if i == 1 or i == 4 or i == 6 or i == 9 or i == 11 or i == 13 or i == 16 or i == 18 or i == 20 or i == 23 or i == 25 or i == 27 or i == 30:
                output.append(x)
        return output

In [ ]:
m = Vgg16(True).cuda()

In [ ]:
m.features

# Helper

In [ ]:
def make_layers(cfg, use_bias, batch_norm=False):
    layers = []
    in_channels = 3
    outputs = []
    for i in range(len(cfg)):
        if cfg[i] == 'O':
            outputs.append(nn.Sequential(*layers))
        elif cfg[i] == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, cfg[i], kernel_size=3, padding=1, bias=use_bias)
            torch.nn.init.xavier_uniform_(conv2d.weight)
            if batch_norm and cfg[i + 1] != 'M':
                layers += [conv2d, nn.BatchNorm2d(cfg[i]), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = cfg[i]
    return nn.Sequential(*layers)

In [ ]:
def make_arch(idx, cfg, use_bias, batch_norm=False):
    return VGG(make_layers(cfg[idx], use_bias, batch_norm=batch_norm))

In [ ]:
def get_networks(config, load_checkpoint=False):
    equal_network_size = config['equal_network_size']
    pretrain = config['pretrain']
    experiment_name = config['experiment_name']
    dataset_name = config['dataset_name']
    normal_class = config['normal_class']
    use_bias = config['use_bias']
    cfg = {
        'A': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
        'B': [16, 16, 'M', 16, 128, 'M', 16, 16, 256, 'M', 16, 16, 512, 'M', 16, 16, 512, 'M'],
    }

    if equal_network_size:
        config_type = 'A'
    else:
        config_type = 'B'

    vgg = Vgg16(pretrain).cuda()
    model = make_arch(config_type, cfg, use_bias, True).cuda()

    for j, item in enumerate(nn.ModuleList(model.features)):
        print('layer : {} {}'.format(j, item))

    if load_checkpoint:
        last_checkpoint = config['last_checkpoint']
        checkpoint_path = "./outputs/{}/{}/checkpoints/".format(experiment_name, dataset_name)
        model.load_state_dict(
            torch.load('{}Cloner_{}_epoch_{}.pth'.format(checkpoint_path, normal_class, last_checkpoint)))
        if not pretrain:
            vgg.load_state_dict(
                torch.load('{}Source_{}_random_vgg.pth'.format(checkpoint_path, normal_class)))
    elif not pretrain:
        checkpoint_path = "./outputs/{}/{}/checkpoints/".format(experiment_name, dataset_name)
        Path(checkpoint_path).mkdir(parents=True, exist_ok=True)

        torch.save(vgg.state_dict(), '{}Source_{}_random_vgg.pth'.format(checkpoint_path, normal_class))
        print("Source Checkpoint saved!")

    return vgg, model

# hyper

In [ ]:
root_save = "./best_models"
root_ds = "./MNIST"

device = 'cuda'

batch_size = 64
img_size = 32
latent_dim = 100
channels = 1
num_epochs = 1000
sample_interval = 1
lr = 0.002
n_critic = 5
lambda_gp =10

# Data

In [ ]:
trans=transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(img_size),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize((0.5,) , (0.5,))
])

train_data=datasets.MNIST(root=root_ds,
                          train=True, transform=trans, download=True)
test_data=datasets.MNIST(root=root_ds,
                          train=False, transform=trans, download=True)

In [ ]:
def get_index(class_num, dataset, mode:str):
    if mode == 'train':
        class_indx = torch.nonzero(
                dataset.train_labels == class_num * torch.ones_like(dataset.train_labels)
        )

    if mode == 'test':
        class_indx = torch.nonzero(
                dataset.test_labels == class_num * torch.ones_like(dataset.test_labels)
        )

    class_indx = class_indx.squeeze(1)
    return class_indx

In [ ]:
train_sampler = SubsetRandomSampler(get_index(8, train_data, 'train'))
test_sampler = SubsetRandomSampler(get_index(8, test_data, 'test'))

train_loader=DataLoader(train_data, batch_size= batch_size, sampler=train_sampler)
test_loader=DataLoader(test_data, batch_size= len(test_sampler), sampler=test_sampler)

# Losses

In [ ]:
class DirectionOnlyLoss(nn.Module):
    def __init__(self):
        super(DirectionOnlyLoss, self).__init__()
        self.similarity_loss = torch.nn.CosineSimilarity()

    def forward(self, output_pred, output_real):
        y_pred_0, y_pred_1, y_pred_2, y_pred_3 = output_pred[3], output_pred[6], output_pred[9], output_pred[12]
        y_0, y_1, y_2, y_3 = output_real[3], output_real[6], output_real[9], output_real[12]

        loss_0 = torch.mean(1 - self.similarity_loss(y_pred_0.view(y_pred_0.shape[0], -1), y_0.view(y_0.shape[0], -1)))
        loss_1 = torch.mean(1 - self.similarity_loss(y_pred_1.view(y_pred_1.shape[0], -1), y_1.view(y_1.shape[0], -1)))
        loss_2 = torch.mean(1 - self.similarity_loss(y_pred_2.view(y_pred_2.shape[0], -1), y_2.view(y_2.shape[0], -1)))
        loss_3 = torch.mean(1 - self.similarity_loss(y_pred_3.view(y_pred_3.shape[0], -1), y_3.view(y_3.shape[0], -1)))

        total_loss = loss_0 + loss_1 + loss_2 + loss_3

        return

In [ ]:
class MseDirectionLoss(nn.Module):
    def __init__(self, lamda):
        super(MseDirectionLoss, self).__init__()
        self.lamda = lamda
        self.criterion = nn.MSELoss()
        self.similarity_loss = torch.nn.CosineSimilarity()

    def forward(self, output_pred, output_real):
        y_pred_0, y_pred_1, y_pred_2, y_pred_3 = output_pred[3], output_pred[6], output_pred[9], output_pred[12]
        y_0, y_1, y_2, y_3 = output_real[3], output_real[6], output_real[9], output_real[12]

        # different terms of loss
        abs_loss_0 = self.criterion(y_pred_0, y_0)
        loss_0 = torch.mean(1 - self.similarity_loss(y_pred_0.view(y_pred_0.shape[0], -1), y_0.view(y_0.shape[0], -1)))
        abs_loss_1 = self.criterion(y_pred_1, y_1)
        loss_1 = torch.mean(1 - self.similarity_loss(y_pred_1.view(y_pred_1.shape[0], -1), y_1.view(y_1.shape[0], -1)))
        abs_loss_2 = self.criterion(y_pred_2, y_2)
        loss_2 = torch.mean(1 - self.similarity_loss(y_pred_2.view(y_pred_2.shape[0], -1), y_2.view(y_2.shape[0], -1)))
        abs_loss_3 = self.criterion(y_pred_3, y_3)
        loss_3 = torch.mean(1 - self.similarity_loss(y_pred_3.view(y_pred_3.shape[0], -1), y_3.view(y_3.shape[0], -1)))

        total_loss = loss_0 + loss_1 + loss_2 + loss_3 + self.lamda * (
                abs_loss_0 + abs_loss_1 + abs_loss_2 + abs_loss_3)

        return total_loss

# Train

In [ ]:
config = {'equal_network_size':False,
'pretrain':True,
'experiment_name':'local_equal_net',
'dataset_name':'mnist',
'normal_class':1,
'use_bias':False,
'equal_network_size':False}

In [ ]:
continue_train = False
os.environ['TORCH_HOME'] = './pretrain_models'
if continue_train:
    vgg, model = get_networks(config, load_checkpoint=True)
else:
    vgg, model = get_networks(config)

In [16]:
# Criteria And Optimizers
direction_loss_only = False
lamda = 0.01

if direction_loss_only:
    criterion = DirectionOnlyLoss()
else:
    criterion = MseDirectionLoss(lamda)

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [21]:
criterion(output_pred, output_real)

tensor(3.4501, device='cuda:0', grad_fn=<AddBackward0>)

In [40]:
vgg.features

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [42]:
vgg(X)

[tensor([[[[0.3087, 0.2984, 0.2984,  ..., 0.2984, 0.2984, 0.3910],
           [0.3020, 0.3569, 0.3569,  ..., 0.3569, 0.3569, 0.4696],
           [0.3020, 0.3569, 0.3569,  ..., 0.3569, 0.3569, 0.4696],
           ...,
           [0.3020, 0.3569, 0.3569,  ..., 0.3569, 0.3569, 0.4696],
           [0.3020, 0.3569, 0.3569,  ..., 0.3569, 0.3569, 0.4696],
           [0.1748, 0.3039, 0.3039,  ..., 0.3039, 0.3039, 0.3423]],
 
          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 2.0592],
           [0.0000, 0.3534, 0.3534,  ..., 0.3534, 0.3534, 2.7446],
           [0.0000, 0.3534, 0.3534,  ..., 0.3534, 0.3534, 2.7446],
           ...,
           [0.0000, 0.3534, 0.3534,  ..., 0.3534, 0.3534, 2.7446],
           [0.0000, 0.3534, 0.3534,  ..., 0.3534, 0.3534, 2.7446],
           [0.2292, 2.1925, 2.1925,  ..., 2.1925, 2.1925, 3.5542]],
 
          [[1.8644, 2.5672, 2.5672,  ..., 2.5672, 2.5672, 4.0903],
           [0.5682, 0.4689, 0.4689,  ..., 0.4689, 0.4689, 1.6727],
           [0.5682, 0.46

In [35]:
X.shape

torch.Size([64, 3, 32, 32])

In [25]:
output_real = vgg(X)
output_real[3].shape

torch.Size([64, 128, 8, 8])

In [20]:
losses = []
for epoch in range(num_epochs + 1):
        model.train()
        epoch_loss = 0
        for data in train_loader:
            X = data[0]
            if X.shape[1] == 1:
                X = X.repeat(1, 3, 1, 1)
            X = Variable(X).cuda()

            output_pred = model.forward(X)
            output_real = vgg(X)

            total_loss = criterion(output_pred, output_real)

            # Add loss to the list
            epoch_loss += total_loss.item()
            losses.append(total_loss.item())

            # Clear the previous gradients
            optimizer.zero_grad()
            # Compute gradients
            total_loss.backward()
            # Adjust weights
            optimizer.step()
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, epoch_loss))

KeyboardInterrupt: 

In [27]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [48]:
model.forward(X)[12].flatten(1).mean(1).shape

torch.Size([64])

In [29]:
count_parameters(model)

343984

In [31]:
X

tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1.